In [2]:
## load libraries
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from collections import Counter 
import plotly.graph_objs as go
from plotly import tools
import seaborn as sns
import pandas as pd 
import numpy as np 
import plotly 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from eli5.sklearn import PermutationImportance
from sklearn.metrics import accuracy_score
import eli5


init_notebook_mode(connected=True)

#load data and remove(students,not employed)
df = pd.read_csv("/kaggle/input/kaggle-survey-2019/multiple_choice_responses.csv",low_memory = False)
df = df[~df['Q5'].isin(['Student','Not employed'])]


In [3]:
df

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...
1,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,83,55-59,Female,-1,Germany,Professional degree,NaN,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,Azure SQL Database,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19711,85,22-24,Male,-1,India,Bachelor’s degree,Data Scientist,-1,50-249 employees,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
19713,84,50-54,Male,-1,Japan,NaN,NaN,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
19714,473,18-21,Male,-1,India,Bachelor’s degree,Other,-1,250-999 employees,3-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
19716,176,25-29,Male,-1,India,Master’s degree,Statistician,-1,50-249 employees,15-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


## 1.Sources of Learning Data Science
Data science skills are straightforward to obtain,they need experience and learning.nowadays there are many online and offline which teaches them in detail.while some prefer online courses such as Coursera,Udacity.Some prefer to go to universities for a year or two-year long dedicated courses.Let's look at what are the most popular sources of learning data science among the respondents of the kaggle-survey. In this question,one participant could have chosen multiple choices, hence the x-axis, represents "percentage" of respondents who selected a particular choice.

In [13]:
## function to identify value counts of a column responses presented in multiple other columns
def summate_columns(col, xaxis_def=None):
    xaxis, yaxis1 = [], []
    for i in range(1, 20):
        if col+'_Part_'+str(i) not in df:
            break
        
        doc = dict(df[col+'_Part_'+str(i)].value_counts())
        if len(doc) > 0:
            key = list(doc.keys())[0]
        else:
            continue 
        xaxis.append(key)
        
        if key in doc:
            yaxis1.append(doc[key])
        else:
            yaxis1.append(0)
            
    if xaxis_def != None:
        xaxis = xaxis_def
    
    xaxis = xaxis[::-1]
    xaxis = [_.split("(")[0]+"    " for _ in xaxis]
    yaxis1 = [x*100/len(df) for x in yaxis1][::-1]

    temp_df = pd.DataFrame()
    temp_df['xaxis'] = xaxis
    temp_df['yaxis'] = yaxis1
    temp_df = temp_df.sort_values("yaxis")
    temp_df['colors'] = ['#fad46b']*len(temp_df)
    colors = np.array(temp_df['colors'])
    colors[-3], colors[-2], colors[-1] = "#97c6e8", "#97c6e8", "#97c6e8"
    colors[-4] = "red"
    
    annotations = []
    for yd, xd in zip(xaxis, yaxis1):
        annotations.append(dict(xref='x', yref='y',
                            x=xd+2, y=yd,
                            text=str(round(xd, 1)) + '%',
                            font=dict(family='Arial', size=14, color='black'),
                            showarrow=False))
    
    fig = go.Figure(data=[go.Bar(name='', y=temp_df['xaxis'], x=temp_df['yaxis'], orientation='h', marker=dict(color=colors, opacity=0.6))], 
                    layout = go.Layout(title="Sources of Learning Data Science", plot_bgcolor='#fff', paper_bgcolor='#fff', margin=dict(l=100), 
                                       width=900, height=500, legend=dict(orientation="h", x=0.1, y=1.1)))
    fig.update_layout(barmode='group')
    
    annotations += [go.layout.Annotation(x=40, y=9, xref="x", yref="y", text="(Most Popular)", showarrow=False, arrowhead=7, ax=0, ay=-40)]
    
    fig.update_layout(annotations=annotations)
    fig.update_xaxes(showgrid=False, zeroline=False, title="% of respondents")
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.add_shape(go.layout.Shape(type="rect", xref="x", yref="paper", x0=22, x1=48.6, y0=0.75, y1=1, fillcolor="gray", opacity=0.2, layer="below", line_width=0))
    fig.show()
    
summate_columns("Q13")